In [3]:
import pandas as pd
import numpy as np

In [4]:
stock = pd.read_csv('AAPL.csv', index_col="Date", parse_dates=True)

In [36]:
stock.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Close_SMA5',
       'Close_SMA20', 'Close_SMA60', 'Close_SMA120', 'Close_SMA180',
       'Open_SMA5', 'Open_SMA20', 'Open_SMA60', 'Open_SMA120', 'Open_SMA180',
       'High_SMA5', 'High_SMA20', 'High_SMA60', 'High_SMA120', 'High_SMA180',
       'Low_SMA5', 'Low_SMA20', 'Low_SMA60', 'Low_SMA120', 'Low_SMA180',
       'Adj Close_SMA5', 'Adj Close_SMA20', 'Adj Close_SMA60',
       'Adj Close_SMA120', 'Adj Close_SMA180', 'Close_EMA5', 'Close_EMA20',
       'Close_EMA60', 'Close_EMA120', 'Close_EMA180', 'Open_EMA5',
       'Open_EMA20', 'Open_EMA60', 'Open_EMA120', 'Open_EMA180', 'High_EMA5',
       'High_EMA20', 'High_EMA60', 'High_EMA120', 'High_EMA180', 'Low_EMA5',
       'Low_EMA20', 'Low_EMA60', 'Low_EMA120', 'Low_EMA180', 'Adj Close_EMA5',
       'Adj Close_EMA20', 'Adj Close_EMA60', 'Adj Close_EMA120',
       'Adj Close_EMA180', 'RSI6', 'RSI14', 'RSI24', 'MACD', 'DIF', 'OSC_Hist',
       'Upper_Band', 'Middle_Band', 'Lowe

In [37]:
stock['RSI14_over_heat'] = stock['RSI14'] > 70
stock['RSI14_over_heat']

Date
1994-07-26    False
1994-07-27    False
1994-07-28    False
1994-07-29     True
1994-08-01    False
              ...  
2024-06-13     True
2024-06-14     True
2024-06-17     True
2024-06-18     True
2024-06-20    False
Name: RSI14_over_heat, Length: 7529, dtype: bool

In [38]:
stock['RSI14_over_cold'] = stock['RSI14'] < 30
stock['RSI14_over_cold']

Date
1994-07-26    False
1994-07-27    False
1994-07-28    False
1994-07-29    False
1994-08-01    False
              ...  
2024-06-13    False
2024-06-14    False
2024-06-17    False
2024-06-18    False
2024-06-20    False
Name: RSI14_over_cold, Length: 7529, dtype: bool

In [39]:
stock['Close > SMA5'] = stock['Close'] > stock['Close_SMA5']
stock['STD > 1.5'] = stock['Close'] .rolling(10).std() > 1.5

In [40]:
stock['Close > SMA5']

Date
1994-07-26     True
1994-07-27     True
1994-07-28     True
1994-07-29     True
1994-08-01     True
              ...  
2024-06-13     True
2024-06-14     True
2024-06-17     True
2024-06-18     True
2024-06-20    False
Name: Close > SMA5, Length: 7529, dtype: bool

In [41]:
stock['STD > 1.5']

Date
1994-07-26    False
1994-07-27    False
1994-07-28    False
1994-07-29    False
1994-08-01    False
              ...  
2024-06-13     True
2024-06-14     True
2024-06-17     True
2024-06-18     True
2024-06-20     True
Name: STD > 1.5, Length: 7529, dtype: bool

In [57]:
# 黃金交叉
def crossover(over,down):
    a1 = over
    b1 = down
    a2 = a1.shift(1)
    b2 = b1.shift(1)
    crossover =  (a1>a2) & (a1>b1) & (b2>a2)
    return crossover
# 死亡交叉
def crossunder(down,over):
    a1 = down
    b1 = over
    a2 = a1.shift(1)
    b2 = b1.shift(1)
    crossdown =  (a1<a2) & (a1<b1) & (b2<a2)
    return crossdown

stock['KD_Golden_Cross'] = crossover(stock['Stochastic_K'],stock['Stochastic_D'])
stock['KD_Death_Cross'] = crossunder(stock['Stochastic_K'],stock['Stochastic_D'])

low_range_kd = 25 # KD值低檔通常設定於20-30之間
stock['Low_Range_D'] = stock['Stochastic_D'] < 25

stock['Buy_In'] = (stock['KD_Golden_Cross'] & stock['Low_Range_D'])
stock['Sold_Out'] = stock['KD_Death_Cross']

In [58]:
stock['MACD_Golden_Cross']=(stock['DIF']>stock['MACD']) & (stock['DIF'].shift(1)<stock['MACD'].shift(1))
stock['MACD_Death_Cross']=(stock['DIF']<stock['MACD']) & (stock['DIF'].shift(1)>stock['MACD'].shift(1))

,Open,High,Low,Close,Adj Close,Volume,Close_SMA5,Close_SMA20,Close_SMA60,Close_SMA120,...,RSI14_over_cold,Close > SMA5,STD > 1.5,KD_Golden_Cross,KD_Death_Cross,Low_Range_D,Buy_In,Sold_Out,MACD_Golden_Cross,MACD_Death_Cross
Date,,,,,,,,,,,,,,,,,,,,,
1994-07-28,0.276786,0.286830,0.275670,0.284598,0.236366,245313600,0.280357,0.253739,0.255262,0.280960,...,False,True,False,False,True,False,False,True,False,False
1994-08-02,0.299107,0.300223,0.289063,0.290737,0.241465,269561600,0.290290,0.263058,0.255476,0.280505,...,False,True,False,False,True,False,False,True,False,False
1994-08-11,0.305804,0.313616,0.302455,0.306362,0.254442,298088000,0.302790,0.281920,0.259177,0.278891,...,False,True,False,False,True,False,False,True,False,False
1994-08-18,0.310268,0.314732,0.308036,0.309152,0.257648,206259200,0.310268,0.297461,0.261958,0.278175,...,False,False,False,False,True,False,False,True,False,False
1994-09-01,0.315848,0.319196,0.309152,0.312500,0.260439,204288000,0.318862,0.310965,0.272087,0.277096,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,195.690002,196.500000,194.169998,194.479996,194.479996,41181800,194.195999,190.222999,178.042333,182.032500,...,False,True,True,False,True,False,False,True,False,True
2024-06-10,196.899994,197.300003,192.149994,193.119995,193.119995,97262100,194.941998,191.342498,178.806999,181.985249,...,False,False,True,False,True,False,False,True,False,True
2024-06-12,207.369995,220.199997,206.899994,213.070007,213.070007,198134300,200.941998,193.667999,180.038333,182.213499,...,False,True,True,False,True,False,False,True,False,True
